### 서버와 클라이언트
서버 : kakao api
클라이언트 :  파이썬(나)

In [1]:
KAKAO_API_KEY = '카카오 rest api키' # Rest Api Key
API_URL='https://dapi.kakao.com/v2/local/search/keyword.json'

In [2]:
import pandas as pd
import numpy as np
import urllib.request # 없으면 설치해야 한다. pip(pip3) install urllib
import json

In [3]:
crime_anal_police = pd.read_csv('./data/02. crime_in_Seoul.csv', thousands=',', encoding='euc-kr')
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


# 경찰서가 무슨 구에 있나?
* 중부서 -> 중구
* 종로서 -> 종로구

### 검색어 설정을 위한 문자열 편집
* 중부서 -> 서울중부경찰서
* 종로서 -> 서울종로경차서
* 남대문서 -> 서울 남대문 경찰서

In [4]:
station_name = []

for name in crime_anal_police['관서명']:
    # print('서울{}경찰서'.format(name[ : -1]))
    station_name.append('서울{}경찰서'.format(name[ : -1]))

station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

## 카카오 api에서 검색어를 전달하기 위한 검색어 인코딩 작업

kakao api에 전달 가능한 형태로 문자열 인코딩하기
* urllib.parse.quote 함수를 활용한다

In [5]:
encText = urllib.parse.quote('서울광진경찰서')
encText

'%EC%84%9C%EC%9A%B8%EA%B4%91%EC%A7%84%EA%B2%BD%EC%B0%B0%EC%84%9C'

In [6]:
request = urllib.request.Request(API_URL+"?query={}".format(encText)) # 카카오 API에 검색까지 포함된 요청을 할 수 있는 요청객체 생성

# kakao에 요청 시에 API_KEY 를 넣지 않으면 인증이 안된 사용자라는 메세지가 뜬다.( UnAuthorization 메세지가 뜬다.)
request.add_header("Authorization", "KakaoAK {}".format(KAKAO_API_KEY)) # 요청시 카카오 API에 나의 인증정보를 헤더에 전달

response = urllib.request.urlopen(request) # 요청 수행 후 카카오 서버로부터 응답 받기

# read() 요청된 내용을 모두 불러오는 역할 -> 한번 불러오면 다시 불러 올 수 없다. (다시 요청 해야 한다.)
# json.loads(response.read()) # 응답받은 내용 확인 ( json 형태로 변환했기 때문에, dictionary 형태로 사용할 수 있다. )

### 필요 데이터 추출
# 1. 경찰서 주소
# 2. 위, 경도
response_json = json.loads(response.read().decode('utf-8')) 
temp_data = response_json['documents'][0]

temp_data

{'address_name': '서울 광진구 구의동 254-32',
 'category_group_code': 'PO3',
 'category_group_name': '공공기관',
 'category_name': '사회,공공기관 > 행정기관 > 경찰서',
 'distance': '',
 'id': '8211607',
 'phone': '02-2285-7602',
 'place_name': '서울광진경찰서',
 'place_url': 'http://place.map.kakao.com/8211607',
 'road_address_name': '서울 광진구 자양로 167',
 'x': '127.08396170505674',
 'y': '37.54292164557882'}

In [7]:
print(temp_data['address_name'])
print(temp_data['y']) # 경도
print(temp_data['x']) # 위도

서울 광진구 구의동 254-32
37.54292164557882
127.08396170505674


## 전체 경찰서에 대한 주소 및 위, 경도 모으기

In [8]:
station_address = [] # 경찰서의 주소를 모을 리스트
station_lat = [] # 위도 -> x
station_lon = [] # 경도 -> y

In [9]:
for name in station_name:
    encText = urllib.parse.quote(name)
    request = urllib.request.Request(API_URL+"?query={}".format(encText)) 
    request.add_header("Authorization", "KakaoAK {}".format(KAKAO_API_KEY))
    
    response = urllib.request.urlopen(request) # 요청 수행 후 카카오 서버로부터 응답 받기
    # 응답 코드 받기 OK : 200, 기타 응답 오류 코드를 수신할 수 있다.
    rescode = response.getcode()
    
    # rescode가 200이면 성공적으로 응답 받았다.
    if rescode == 200:
        #응답을 받고 그 응답에 대한 데이터를 보통 body라고 한다.
        response_body = response.read()
        
        # 응답받은 데이터(response_body)를 utf-8으로 인코딩한 후 json 객체로 만들어 주면
        tmp = json.loads(response_body.decode('utf-8'))
        tmp_doc = tmp['documents']
        tmp_data = tmp_doc[0]
        
        address = tmp_data['address_name']
        lat = temp_data['x']
        lng = temp_data['y']
        
        station_address.append(address) # 리스트에 주소 추가
        station_lat.append(lat) # 리스트에 경도 추가
        station_lon.append(lng) # 리스트에 위도 추가
        
        print('{} --> {}'.format(name,address))
        

서울중부경찰서 --> 서울 중구 저동2가 62-1
서울종로경찰서 --> 서울 종로구 경운동 90-18
서울남대문경찰서 --> 서울 중구 남대문로5가 567
서울서대문경찰서 --> 서울 서대문구 미근동 165
서울혜화경찰서 --> 서울 종로구 인의동 48-57
서울용산경찰서 --> 서울 용산구 원효로1가 12-12
서울성북경찰서 --> 서울 성북구 삼선동5가 301
서울동대문경찰서 --> 서울 동대문구 청량리동 229
서울마포경찰서 --> 서울 마포구 아현동 618-1
서울영등포경찰서 --> 서울 영등포구 당산동3가 2-11
서울성동경찰서 --> 서울 성동구 행당동 192-8
서울동작경찰서 --> 서울 동작구 노량진동 72-35
서울광진경찰서 --> 서울 광진구 구의동 254-32
서울서부경찰서 --> 서울 은평구 녹번동 177-15
서울강북경찰서 --> 서울 강북구 번동 415-15
서울금천경찰서 --> 서울 금천구 시흥동 1030
서울중랑경찰서 --> 서울 중랑구 신내동 810
서울강남경찰서 --> 서울 강남구 대치동 998
서울관악경찰서 --> 서울 관악구 봉천동 1695-5
서울강서경찰서 --> 서울 양천구 신월동 25
서울강동경찰서 --> 서울 강동구 성내동 541-1
서울종암경찰서 --> 서울 성북구 종암동 3-1260
서울구로경찰서 --> 서울 구로구 구로동 436
서울서초경찰서 --> 서울 서초구 서초동 1726-1
서울양천경찰서 --> 서울 양천구 신정동 321
서울송파경찰서 --> 서울 송파구 가락동 9
서울노원경찰서 --> 서울 노원구 하계동 250
서울방배경찰서 --> 서울 서초구 방배동 455-10
서울은평경찰서 --> 서울 은평구 불광동 산 24
서울도봉경찰서 --> 서울 도봉구 창동 17
서울수서경찰서 --> 서울 강남구 개포동 14


In [10]:
 station_address[:5] # 5개만 확인

['서울 중구 저동2가 62-1',
 '서울 종로구 경운동 90-18',
 '서울 중구 남대문로5가 567',
 '서울 서대문구 미근동 165',
 '서울 종로구 인의동 48-57']

In [11]:
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


# 경찰서 위치에 대한 구별 데이터 추가
* split()으로 문자열을 자르고 인덱스 1번을 -구로 삼게 되면
    * 서울특별시 -구(o)
    * 서울 -구(o)
    * 서울 특별시 -구(x)
    
* split('구')로 자르고 난 다음 편집
    * 구의 이름만 등장한다. -> [서울 특별시 광진, 가] -> [-1] 쓰면 된다.
    * 문제 발생 상황 : 구로구(x)
    
<br>
문자열 데이터에서 내가 필요한 정보만 가지고 올 수 있게 하는 것 -> 텍스트 마이닝

우리가 하고 있는 건 전처리 알고리즘을 직접 생각하고 나서 할 수 있는 텍스트 마이닝

In [12]:
gu_name = []

for name in station_address:
    tmp = name.split() # 공백을 기준으로 자른다.
    
    # 구로 끝나는 부분만 추출할 수 있는 정규식을 만들어서 사용할 수도 있다.
    tmp_gu = [gu for gu in tmp if gu[-1] == '구'] # 공백을 기준으로 잘린 리스트를 검사해서 끝이 '구'로 끝나는 부분만 리스트에 추가
    print(tmp_gu[0])
    
    gu_name.append(tmp_gu[0]) # 구의 이름 추가

중구
종로구
중구
서대문구
종로구
용산구
성북구
동대문구
마포구
영등포구
성동구
동작구
광진구
은평구
강북구
금천구
중랑구
강남구
관악구
양천구
강동구
성북구
구로구
서초구
양천구
송파구
노원구
서초구
은평구
도봉구
강남구


# 각 경찰서에 대한 구의 추출 후 데이터 프레임에 추가

In [13]:
crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구


# 메인 주제 : 구별 범죄 발생, 범죄 검거에 대한 분석

In [14]:
# 구별 데이터가 잘 수집 되었는지 확인이 필요
crime_anal_police['구별'].unique()

array(['중구', '종로구', '서대문구', '용산구', '성북구', '동대문구', '마포구', '영등포구', '성동구',
       '동작구', '광진구', '은평구', '강북구', '금천구', '중랑구', '강남구', '관악구', '양천구',
       '강동구', '구로구', '서초구', '송파구', '노원구', '도봉구'], dtype=object)

In [15]:
len(crime_anal_police['구별'].unique()) # 서울시의 구는 25개의 행정구가 존재하는데 실제 수집된건 24개

24

데이터 분석에서는 내가 분석하고자 하는 대상의 비즈니스를 잘 알고 있는 것도 중요하다.
* 데이터 분석 전 자료조사가 필요할 수도 있다.
* 설문조사, 논문, 검색 등을 참고한다.

### 데이터 분석에서 제일 시간이 많이 걸리는 것 중에 하나가 비즈니스 분석

In [16]:
# 20200713 기준 서울강서경찰서는 양천구에 임시청사가 있다.
crime_anal_police.loc[crime_anal_police['관서명']=='강서서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
19,강서서,7,8,13,13,262,191,2096,1260,3207,2718,양천구


In [17]:
crime_anal_police.loc[crime_anal_police['관서명']=='강서서','구별'] = '강서구'

In [18]:
crime_anal_police

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구
5,용산서,5,5,14,14,194,173,1557,587,2050,1704,용산구
6,성북서,2,2,2,1,86,71,953,409,1194,1015,성북구
7,동대문서,5,5,13,13,173,146,1981,814,2548,2227,동대문구
8,마포서,8,8,14,10,294,247,2555,813,2983,2519,마포구
9,영등포서,14,12,22,20,295,183,2964,978,3572,2961,영등포구


In [19]:
crime_anal_police.loc[crime_anal_police['관서명']=='강서서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
19,강서서,7,8,13,13,262,191,2096,1260,3207,2718,강서구


강서서를 강서구로 바꾸었음을 확인할 수 있다.

In [20]:
crime_anal_police.to_csv('./data/crime_anal_police_gu.csv',encoding='utf-8')

# 경찰서가 31개, 서울시의 구는 25개
구별로 분석을 해야 함. => 구별로 집계가 필요

In [21]:
crime_anal_raw = crime_anal_police.copy()
crime_anal_raw

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구
5,용산서,5,5,14,14,194,173,1557,587,2050,1704,용산구
6,성북서,2,2,2,1,86,71,953,409,1194,1015,성북구
7,동대문서,5,5,13,13,173,146,1981,814,2548,2227,동대문구
8,마포서,8,8,14,10,294,247,2555,813,2983,2519,마포구
9,영등포서,14,12,22,20,295,183,2964,978,3572,2961,영등포구


# 구별 범죄 발생 횟수의 합, 범죄 검거 횟수의 합

In [22]:
crime_anal = pd.pivot_table(crime_anal_raw,index=['구별'],aggfunc=np.sum)
crime_anal

,강간 검거,강간 발생,강도 검거,강도 발생,살인 검거,살인 발생,절도 검거,절도 발생,폭력 검거,폭력 발생
구별,,,,,,,,,,
강남구,349,449,18,21,10,13,1650,3850,3705,4284
강동구,123,156,8,6,3,4,789,2366,2248,2712
강북구,126,153,13,14,8,7,618,1434,2348,2649
강서구,191,262,13,13,8,7,1260,2096,2718,3207
관악구,221,320,14,12,8,9,827,2706,2642,3298
광진구,220,240,26,14,4,4,1277,3026,2180,2625
구로구,164,281,11,15,6,8,889,2335,2432,3007
금천구,122,151,6,6,4,3,888,1567,1776,2054
노원구,121,197,7,7,10,10,801,2193,2329,2723


In [23]:
crime_anal['강간검거율'] = crime_anal['강간 검거'] / crime_anal['강간 발생']*100
crime_anal['강도검거율'] = crime_anal['강도 검거'] / crime_anal['강도 발생']*100
crime_anal['살인검거율'] = crime_anal['살인 검거'] / crime_anal['살인 발생']*100
crime_anal['절도검거율'] = crime_anal['절도 검거'] / crime_anal['절도 발생']*100
crime_anal['폭력검거율'] = crime_anal['폭력 검거'] / crime_anal['폭력 발생']*100

# 검거율을 사용할 것이기 때문에 검거량에 대한 데이터는 삭제
del crime_anal['강간 검거']
del crime_anal['강도 검거']
del crime_anal['살인 검거']
del crime_anal['절도 검거']
del crime_anal['폭력 검거']

crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,133.333333,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,114.285714,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,114.285714,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,116.666667,88.888889,30.561715,80.109157


전년도 범죄를 검거 한 경우 100%가 넘을 수 있다. -> 그냥 100%로 바꾸기

In [24]:
con_list = ['강간검거율','강도검거율','살인검거율','절도검거율','폭력검거율']

# 각 검거율 컬럼에 대해 100이 넘어가는 값은 100으로 통일
for column in con_list:
    crime_anal.loc[crime_anal[column]>100,column] = 100

crime_anal

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,100.000000,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,100.000000,88.888889,30.561715,80.109157
광진구,240,14,4,3026,2625,91.666667,100.000000,100.000000,42.200925,83.047619
구로구,281,15,8,2335,3007,58.362989,73.333333,75.000000,38.072805,80.877951
금천구,151,6,3,1567,2054,80.794702,100.000000,100.000000,56.668794,86.465433
노원구,197,7,10,2193,2723,61.421320,100.000000,100.000000,36.525308,85.530665


In [25]:
crime_anal.rename(columns={
    '강간 발생':'강간',
    '강도 발생':'강도',
    '살인 발생':'살인',
    '절도 발생':'절도',
    '폭력 발생':'폭력'},inplace=True)

crime_anal.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,100.000000,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,100.000000,88.888889,30.561715,80.109157


# 데이터 정규화(normalization)
정규화의 목적
* 정규화의 종류
    * MinMax 정규화 : 데이터의 편차를 없애기 위해 사용. 분석하고자 하는 대상의 데이터들의 편차가 클 때 사용
    * Standard 정규화 : 데이터들의 평균을 0으로, 분산을 1. 데이터의 분포를 일정하게 할 때 사용
    * Robust 정규화 : 이상치를 제거하기 위한 목적으로 사용
    * 벡터 정규화 : 데이터의 방향성이 중요할 때 사용(어떠한 데이터가 닮았나...각도가 중요할 때)
    <br><br>
* 정규화의 목적
    * 컴퓨터든, 사람이든 데이터를 놓고 봤을 때 이해하기 쉬워지도록 데이터를 편집

# sklearn을 활용해 정규화 하기

In [26]:
from sklearn import preprocessing # 사이킷런

# 정규화 대상 컬럼 선정
cols = ['강간','강도','살인','절도','폭력']

# 정규화 진행시킬 컬럼, 데이터 가져오기
x = crime_anal[cols].values
x

array([[ 449,   21,   13, 3850, 4284],
       [ 156,    6,    4, 2366, 2712],
       [ 153,   14,    7, 1434, 2649],
       [ 262,   13,    7, 2096, 3207],
       [ 320,   12,    9, 2706, 3298],
       [ 240,   14,    4, 3026, 2625],
       [ 281,   15,    8, 2335, 3007],
       [ 151,    6,    3, 1567, 2054],
       [ 197,    7,   10, 2193, 2723],
       [ 102,    9,    3, 1063, 1487],
       [ 173,   13,    5, 1981, 2548],
       [ 285,    9,    5, 1865, 1910],
       [ 294,   14,    8, 2555, 2983],
       [ 154,    5,    2, 1812, 2056],
       [ 393,    9,    8, 2635, 2399],
       [ 126,    9,    4, 1607, 1612],
       [ 150,    5,    5, 1785, 2209],
       [ 220,   13,   11, 3239, 3295],
       [ 120,    6,    3, 1890, 2509],
       [ 295,   22,   14, 2964, 3572],
       [ 194,   14,    5, 1557, 2050],
       [ 166,    9,    3, 1914, 2653],
       [ 211,   11,    6, 2184, 2293],
       [ 170,    9,    3, 2548, 2224],
       [ 187,   11,   13, 2135, 2847]], dtype=int64)

In [27]:
# MinMaxScaler 가져오기
min_mac_scaler = preprocessing.MinMaxScaler()

# 정규화 진행
x_scaled = min_mac_scaler.fit_transform(x.astype(float)) # 2차원 배열 x에 들어가있는 값들을 실수형으로 바꾼 다음 스케일링 진행 
x_scaled
# 0은 가장 적게 발생했다는 뜻(없다는 뜻이 아니다.)

array([[1.        , 0.94117647, 0.91666667, 1.        , 1.        ],
       [0.1556196 , 0.05882353, 0.16666667, 0.46752781, 0.43796925],
       [0.14697406, 0.52941176, 0.41666667, 0.13311805, 0.41544512],
       [0.4610951 , 0.47058824, 0.41666667, 0.37064944, 0.61494458],
       [0.62824207, 0.41176471, 0.58333333, 0.58952278, 0.64747944],
       [0.39769452, 0.52941176, 0.16666667, 0.70434159, 0.4068645 ],
       [0.51585014, 0.58823529, 0.5       , 0.45640474, 0.5434394 ],
       [0.14121037, 0.05882353, 0.08333333, 0.18083961, 0.2027172 ],
       [0.27377522, 0.11764706, 0.66666667, 0.40545389, 0.44190204],
       [0.        , 0.23529412, 0.08333333, 0.        , 0.        ],
       [0.20461095, 0.47058824, 0.25      , 0.32938644, 0.379335  ],
       [0.52737752, 0.23529412, 0.25      , 0.28776462, 0.15123346],
       [0.55331412, 0.52941176, 0.5       , 0.53534266, 0.53485878],
       [0.14985591, 0.        , 0.        , 0.26874776, 0.20343225],
       [0.83861671, 0.23529412, 0.

In [28]:
# 정규화된 범죄 건수가 들어있는 새로운 데이터 프레임 만들기
crime_anal_norm = pd.DataFrame(x_scaled,columns=cols, index=crime_anal.index)
crime_anal_norm.head()

,강간,강도,살인,절도,폭력
구별,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000
강동구,0.155620,0.058824,0.166667,0.467528,0.437969
강북구,0.146974,0.529412,0.416667,0.133118,0.415445
강서구,0.461095,0.470588,0.416667,0.370649,0.614945
관악구,0.628242,0.411765,0.583333,0.589523,0.647479


In [29]:
col2 = ['강간검거율','강도검거율','살인검거율','절도검거율','폭력검거율']
crime_anal_norm[col2] = crime_anal[col2]
crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,0.155620,0.058824,0.166667,0.467528,0.437969,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,0.146974,0.529412,0.416667,0.133118,0.415445,82.352941,92.857143,100.000000,43.096234,88.637222
강서구,0.461095,0.470588,0.416667,0.370649,0.614945,72.900763,100.000000,100.000000,60.114504,84.752105
관악구,0.628242,0.411765,0.583333,0.589523,0.647479,69.062500,100.000000,88.888889,30.561715,80.109157


# CCTV, 인구수 데이터까지 포함하기

In [30]:
result_CCTV = pd.read_csv('./data/pop_cctv_row.csv',encoding='utf-8',index_col='구별')
result_CCTV.head()

,Unnamed: 0,소계,2013년도 이전,2014년,2015년,2016년,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,,,,,,
강남구,0,2780,1292,430,584,932,150.619195,570500.0,565550.0,4950.0,63167.0,0.867660,11.072217
강동구,1,773,379,99,155,377,166.490765,453233.0,449019.0,4214.0,54622.0,0.929765,12.051638
강북구,2,748,369,120,138,204,125.203252,330192.0,326686.0,3506.0,54813.0,1.061806,16.600342
강서구,3,884,388,258,184,81,134.793814,603772.0,597248.0,6524.0,72548.0,1.080540,12.015794
관악구,4,1496,846,260,390,613,149.290780,525515.0,507203.0,18312.0,68082.0,3.484582,12.955291


In [31]:
# 소계와 인구수만 가져와서 추가해줄 예정
crime_anal_norm[['인구수','CCTV']] = result_CCTV[['인구수','소계']]
crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,인구수,CCTV
구별,,,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000,77.728285,85.714286,76.923077,42.857143,86.484594,570500.0,2780
강동구,0.155620,0.058824,0.166667,0.467528,0.437969,78.846154,100.000000,75.000000,33.347422,82.890855,453233.0,773
강북구,0.146974,0.529412,0.416667,0.133118,0.415445,82.352941,92.857143,100.000000,43.096234,88.637222,330192.0,748
강서구,0.461095,0.470588,0.416667,0.370649,0.614945,72.900763,100.000000,100.000000,60.114504,84.752105,603772.0,884
관악구,0.628242,0.411765,0.583333,0.589523,0.647479,69.062500,100.000000,88.888889,30.561715,80.109157,525515.0,1496


# 범죄 발생에 대한 합 구하기
* 어떠한 구에 어떤 범죄가 많이 일어나느냐 -> 정규화를 사용해서 구했다.
* 어떠한 구에 범죄 자체가 많이 일어나느냐 -> 범죄율의 합으로 구할 수 있다.

In [32]:
col = ['강간','강도','살인','절도','폭력']
crime_anal_norm['범죄'] = np.sum(crime_anal_norm[col],axis=1) # 정규화된 범죄 발생 건수를 합하여 범죄 칼럼 구하기
# axis가 0이면 행방향, 1이면 열방향
crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,인구수,CCTV,범죄
구별,,,,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000,77.728285,85.714286,76.923077,42.857143,86.484594,570500.0,2780,4.857843
강동구,0.155620,0.058824,0.166667,0.467528,0.437969,78.846154,100.000000,75.000000,33.347422,82.890855,453233.0,773,1.286607
강북구,0.146974,0.529412,0.416667,0.133118,0.415445,82.352941,92.857143,100.000000,43.096234,88.637222,330192.0,748,1.641616
강서구,0.461095,0.470588,0.416667,0.370649,0.614945,72.900763,100.000000,100.000000,60.114504,84.752105,603772.0,884,2.333944
관악구,0.628242,0.411765,0.583333,0.589523,0.647479,69.062500,100.000000,88.888889,30.561715,80.109157,525515.0,1496,2.860342


# 검거율에 대한 합 구하기

In [33]:
col = ['강간검거율','강도검거율','살인검거율','절도검거율','폭력검거율']
crime_anal_norm['검거'] = np.sum(crime_anal_norm[col],axis=1) # 정규화된 범죄 발생 건수를 합하여 범죄 칼럼 구하기

crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,인구수,CCTV,범죄,검거
구별,,,,,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000,77.728285,85.714286,76.923077,42.857143,86.484594,570500.0,2780,4.857843,369.707384
강동구,0.155620,0.058824,0.166667,0.467528,0.437969,78.846154,100.000000,75.000000,33.347422,82.890855,453233.0,773,1.286607,370.084431
강북구,0.146974,0.529412,0.416667,0.133118,0.415445,82.352941,92.857143,100.000000,43.096234,88.637222,330192.0,748,1.641616,406.943540
강서구,0.461095,0.470588,0.416667,0.370649,0.614945,72.900763,100.000000,100.000000,60.114504,84.752105,603772.0,884,2.333944,417.767372
관악구,0.628242,0.411765,0.583333,0.589523,0.647479,69.062500,100.000000,88.888889,30.561715,80.109157,525515.0,1496,2.860342,368.622261


In [34]:
crime_anal_norm.to_csv('./data/crime_anal_norm_final.csv', encoding='utf-8', index_label='구별')